# シグナリングゲームにおける進化的強化学習（ポリアの壺）

このノートブックでは，ポリアの壺を用いた送信者と受信者によるシグナリングゲームにおいて，遺伝的な進化の要素を導入した進化的強化学習の実験を行います．
ここでは遺伝子として，それぞれの壺に最初に入っているボールの数を決める遺伝子を考えます．これは，生まれながらにして本能的に行う行動のバイアスを決める遺伝子に対応します．

それぞれの個体が一生のうちに得られる学習機会が少なかったり，コミュニケーションの失敗が致命的である場合などには，学習に頼らない本能的なコミュニケーションシステムが進化するかもしれません．
ただし，ある遺伝子がコミュニケーションを成功させることに有用かどうかは，集団内の他の個体が持つ遺伝子や学習状況にも依存します．このため，首の長さを決める遺伝子など，個体それぞれで独立して適応度に影響を与える遺伝子よりも，進化の過程は複雑なものになることが予想されます．


In [ ]:
import numpy as np
from tqdm import tqdm
rng = np.random.default_rng()

まず，シグナリングゲームを表現する強化学習環境（`SignalingGameEnv`）を定義します．
これは前回のシグナリングゲーム環境とほとんど同じですが，今回は遺伝的進化を扱うことから， **報酬** と **適応度** を区別する必要があることに注意してください．

- 報酬は，出生後の学習を方向付けるための，個体内部のメカニズムによって与えられるものです．これは脳が出すホルモンなどに対応します．
- 適応度は，次の世代に遺伝子を残すことができるかどうかを決めるための，個体の外部の環境のメカニズムによって決まるものです．

適応度と報酬は，関連するように進化します．例えば，食物を摂取することは適応度を向上させますが，多くの生物は食物を摂取すると「おいしい」と感じる報酬を得て，その後食物の摂取を積極的に行うように学習されます．
しかし，必ずしも適応度と報酬は一致しません．例えば，時々味覚障害をもつ個体が生まれることがありますが，この場合は食物の摂取が適応度を向上させるにもかかわらず，報酬を得られないことがあります．また，多くの哺乳類は子どもを愛おしく思う報酬を得るように進化していますが，これは自分自身の適応度を向上させるわけではなく，むしろ子どもを守るために自分を犠牲にするような行動に報酬を感じたりすることもあります．このように，適応度と報酬は必ずしも一致しないということを考慮することは，進化的強化学習の計算モデルを設計する上で重要です．

ここでは，適応度や報酬をどう設定するかについて，後述するように実験条件としていくつかのパターンを試します．このため，以下の環境の定義では，`rewards` を直接与えるのではなく，`success` という変数で成功したかどうかだけを保持するようにしています．

In [ ]:
class SignalingGameEnv:
    def __init__(self, num_states, num_messages, state_dist):
        self.num_states = num_states
        self.num_messages = num_messages
        self.state_dist = state_dist
        self.agent_names = ['sender', 'receiver']
        self.reset()

    def reset(self):
        self.state = rng.choice(self.num_states, p=self.state_dist)
        self.success = None
        self.done = False
        self.agent_selection = 'sender'

    def observe(self, agent_name):
        if agent_name == 'sender':
            return self.state
        elif agent_name == 'receiver':
            return self.message

    def step(self, action):
        assert self.done == False, 'WARNING: The game is done. Call reset().'
        agent_name = self.agent_selection
        if agent_name == 'sender':
            self.message = action
            self.agent_selection = 'receiver'
        elif agent_name == 'receiver':
            self.action = action
            self.success = self.state == action
            self.done = True

次に，遺伝子を表現するクラス `Gene` を定義します．
ここでは，ポリアの壺に最初に入っているボールの数を表現するための遺伝子を想定しています．
これを表現するために，与えられた `size` の大きさで，それぞれの要素が `min_val` と `max_val` の間の値を取るようなNumpy配列を保持するクラスとして定義します．また，交叉や突然変異のメソッドを定義しています．

In [ ]:
class Gene:
    def __init__(self, size, min_val, max_val, init_values='random'):
        # size は整数または整数のタプルです．
        self.size = size
        self.min_val = min_val
        self.max_val = max_val
        self.init_values = init_values
        if init_values == 'random':
            self.values = rng.uniform(self.min_val, self.max_val, size)
        else:
            self.values = np.full(size, init_values)

    def crossover(self, other_gene):
        new_gene = Gene(self.size, self.min_val, self.max_val, init_values=self.init_values)
        random_mask = rng.integers(0, 2, self.size)
        new_gene.values = np.where(random_mask, self.values, other_gene.values)
        return new_gene

    def mutate(self, mutation_rate):
        random_mask = rng.uniform(0, 1, self.size) < mutation_rate
        self.values = np.where(random_mask, rng.uniform(self.min_val, self.max_val, self.size), self.values)
        return self


次に，ポリアの壺モデルに基づく強化学習エージェントのクラス（`UrnAgent`）を定義します．
これは，前回のノートブックで定義したものとほとんど同じですが，`Gene` クラスのオブジェクトを受け取って初期化するように変更しています．
ここでは `size = (num_obs, num_actions)` で，`values` の $(i, j)$ 要素が壺 $i$ の中にボール $j$ が最初に入っている個数であるような `Gene` クラスのオブジェクトが与えられることを想定しています．

In [ ]:
class UrnAgent:
    def __init__(self, bias_gene):
        self.bias_gene = bias_gene
        self.num_obs = bias_gene.size[0]
        self.num_choices = bias_gene.size[1]
        self.urn_balls = [np.ones(self.num_choices, dtype=float) for _ in range(self.num_obs)]
        self.urn_sum_balls = [self.num_choices for _ in range(self.num_obs)]
        self.train_buf = []
        # Apply innate bias
        for obs in range(self.num_obs):
            self.urn_balls[obs] += self.bias_gene.values[obs]
            self.urn_sum_balls[obs] = self.urn_balls[obs].sum()

    def get_action(self, obs):
        p = self.urn_balls[obs] / self.urn_sum_balls[obs]
        return rng.choice(np.arange(self.num_choices), p=p)

    def store_buffer(self, obs, choice):
        self.train_buf.append([obs, choice, 0]) # 観測，選択，報酬の組

    def update_reward(self, reward):
        if len(self.train_buf) > 0:
          self.train_buf[-1][2] += reward

    def train(self):
        for obs, choice, reward in self.train_buf:
            self.urn_balls[obs][choice] += reward
            self.urn_sum_balls[obs] += reward
        self.train_buf = []

def str_agent(agent):
    str_urn = ''
    for obs in range(agent.num_obs):
        for choice in range(agent.num_choices):
            str_urn += f'{obs}->{choice}:{agent.urn_balls[obs][choice]:.0f}({agent.bias_gene.values[obs][choice]:.0f}), '
        str_urn += '\n'
    return str_urn

また，一つの個体の遺伝子，エージェント，適応度，寿命を保持するためのクラス `Creature` を定義しておきます．
一つの個体は，送信者エージェントと受信者エージェントのペアで構成されるものとして，初期化時にこれらのエージェントを生成します．このため，初期化時に，これらのエージェントの遺伝子 `bias_genes` を引数として受け取ることとしています．`bias_genes` は，`{'sender': Gene, 'receiver': Gene}` という形式の辞書で，それぞれのエージェントの遺伝子を指定することと想定します．

In [ ]:
class Creature:
    def __init__(self, bias_genes, lifespan_mean):
        self.bias_genes = bias_genes
        self.agents = dict()
        for agent_name, bias_gene in bias_genes.items():
            self.agents[agent_name] = UrnAgent(bias_gene)
        self.fitness = 0
        # (1 / lifespan_mean)をパラメータとした幾何分布から寿命をサンプルする
        self.lifespan = rng.geometric(1 / lifespan_mean)

以下のセルで，具体的に環境と初期集団を生成しています．
環境のパラメータを変えて実験してみましょう．

また，ここでは，個体同士のペアの組まれやすさの偏り（近接性）も定義しています．
創発言語の研究では，集団全体でランダムにペアを組むよりも，一部の個体同士が頻繁にペアを組むような近接性がある方が，より効率的にコミュニケーションが創発することが知られています．
また，集団遺伝学におけるマルチレベル選択の理論においても，集団の利益を向上させるような進化において，近接性が重要な役割を果たすことが知られています．
この近接性を定義するために，以下のコードでは，それぞれの送信者が，どの受信者とペアになりやすいかを表す確率を定義しています．
これについても，色々なパターンを変えて実験してみましょう．

In [ ]:
num_states = 4          # 状態数
num_messages = 4        # 信号数
state_dist = [1.0 / num_states] * num_states # 状態の確率分布
env = SignalingGameEnv(num_states, num_messages, state_dist)

pop_size = 20           # 集団内の個体数
num_episodes_per_creature = 100 # 1世代における1個体あたりのエピソード数
min_bias = 0.0          # 遺伝的な行動バイアスの最小値
max_bias = 30.0         # 遺伝的な行動バイアスの最大値
init_bias = 0.0         # 遺伝的な行動バイアスの初期値．乱数で初期化する場合は'random'
success_fitness = {'sender': 0.1, 'receiver': 1} # コミュニケーション成功時の適応度増加量
success_reward = {'sender': 1, 'receiver': 1} # コミュニケーション成功時の報酬量
tornament_size = 10     # トーナメントサイズ
do_crossover = True     # 交叉を行うかどうか．行う場合は親二人，行わない場合は親一人のみを選択
mutation_rate = 0.001   # 突然変異率
lifespan_mean = 1.0     # 平均寿命（幾何分布のパラメータの逆数）

# 近接性の定義
matching_bias = 'group'
num_groups = 5
matching_dists = []
if matching_bias == 'uniform':
    for s in range(pop_size):
        matching_dist = np.ones(pop_size) / pop_size
        matching_dists.append(matching_dist)
elif matching_bias == 'group':
    for s in range(pop_size):
        group = s % num_groups
        matching_dist = np.zeros(pop_size)
        matching_dist[group::num_groups] = 1
        matching_dist /= matching_dist.sum()
        matching_dists.append(matching_dist)

# 初期個体群の生成
population = []
for _ in range(pop_size):
    sender_bias_gene = Gene((num_states, num_messages), min_bias, max_bias, init_bias)
    receiver_bias_gene = Gene((num_messages, num_states), min_bias, max_bias, init_bias)
    bias_genes = {'sender': sender_bias_gene, 'receiver': receiver_bias_gene}
    population.append(Creature(bias_genes, lifespan_mean))

# あとで進化過程を確認するためのログ
log_success_rate = []
log_fitness_mean = []
log_fitness_var = []
log_rewards_mean = {'sender': [], 'receiver': []}
log_bias_suc_fail_diff_mean = {'sender': [], 'receiver': []}

以下のコードで，進化的強化学習のメインループを実装しています．

In [ ]:
num_gen = 1000           # 世代数

# 進化計算
for gen in range(num_gen):
    log_success = []
    log_fitness = []
    log_rewards = {'sender': [], 'receiver': []}
    log_success_choice_biases = {'sender': [], 'receiver': []}
    log_fail_choice_biases = {'sender': [], 'receiver': []}
    # Running episodes
    num_episodes_in_this_gen = num_episodes_per_creature * pop_size
    for episode in range(num_episodes_in_this_gen):
        # 送信者と受信者の選択
        sender_idx = rng.integers(0, pop_size)
        sender = population[sender_idx]
        receiver = rng.choice(population, p=matching_dists[sender_idx])
        creatures = {'sender': sender, 'receiver': receiver}
        env.reset()
        while not env.done:
            agent_name = env.agent_selection
            obs = env.observe(agent_name)
            agent = creatures[agent_name].agents[agent_name]
            action = agent.get_action(obs)
            env.step(action)
            agent.store_buffer(obs, action)
        # 1エピソード終了．成功・不成功に基づいてfitnessとrewardを与え，方策を更新する．
        log_success.append(1 if env.success else 0)
        for _agent_name in env.agent_names:
            # 適応度
            fitness = success_fitness[_agent_name] if env.success else 0
            creatures[_agent_name].fitness += fitness
            # 報酬
            reward = success_reward[_agent_name] if env.success else 0
            creatures[_agent_name].agents[_agent_name].update_reward(reward)
            log_rewards[_agent_name].append(reward)
            # 遺伝子のロギング（成功時に，その行動が遺伝的にどれだけバイアスされていたか）
            if _agent_name == 'sender':
                obs, choice = env.state, env.message
            elif _agent_name == 'receiver':
                obs, choice = env.message, env.action
            choice_bias = creatures[_agent_name].bias_genes[_agent_name].values[obs, choice]
            if env.success:
                log_success_choice_biases[_agent_name].append(choice_bias)
            else:
                log_fail_choice_biases[_agent_name].append(choice_bias)
            # 方策を更新する
            creatures[_agent_name].agents[_agent_name].train()
                

    # 1世代終了．最も適応度の高い個体の壺のボールの数と遺伝子を表示
    best_creature = max(population, key=lambda x: x.fitness)
    print(f'gen {gen} best creature (fitness {best_creature.fitness}):')
    print('sender:')
    print(str_agent(best_creature.agents['sender']))
    print('receiver:')
    print(str_agent(best_creature.agents['receiver']))
    # 適応度のロギング
    log_fitness = [creature.fitness for creature in population]
    # 次世代の個体群を生成する．
    new_population = []
    for i in range(pop_size):
        # 寿命が来ていない個体はそのまま次世代に引き継ぐ
        if population[i].lifespan > 1:
            population[i].lifespan -= 1
            new_population.append(population[i])
            continue
        # トーナメント選択で親を選び，突然変異を行う．
        tournament = rng.choice(population, tornament_size, replace=False)
        if do_crossover:
            winners = sorted(tournament, key=lambda x: x.fitness, reverse=True)[:2]
            new_genes = dict()
            for role in env.agent_names:
                new_genes[role] = winners[0].bias_genes[role].crossover(winners[1].bias_genes[role])
        else:
            winner = max(tournament, key=lambda x: x.fitness)
            new_genes = winner.bias_genes
        for role in env.agent_names:
            new_genes[role].mutate(mutation_rate)
        new_population.append(Creature(new_genes, lifespan_mean))
    population = new_population

    # 進化過程のロギング
    log_success_rate.append(np.mean(log_success))
    for _agent_name in env.agent_names:
        log_fitness_mean.append(np.mean(log_fitness))
        log_fitness_var.append(np.var(log_fitness))
        log_rewards_mean[_agent_name].append(np.mean(log_rewards[_agent_name]))
        suc_bias_mean = np.mean(log_success_choice_biases[_agent_name])
        fail_bias_mean = np.mean(log_fail_choice_biases[_agent_name])
        log_bias_suc_fail_diff_mean[_agent_name].append(suc_bias_mean - fail_bias_mean)

    print(f'gen {gen}: success_rate {log_success_rate[-1]} ' \
          f'fitness_mean {log_fitness_mean[-1]} ' \
          f'fitness_var {log_fitness_var[-1]}')

以下のコードでは，学習過程における報酬のログをTensorboardで確認できる形式で保存しています．
記録している情報は以下の通りです．
- `success/success_rate`: 各世代での成功率のエピソードごとの平均と分散
- `fitness/fitness_{mean,var}`: 各世代での適応度の個体ごとの平均と分散
- `reward/{sender,receiver}_reward_mean`: 各世代での送信者と受信者の報酬のエピソードごとの平均
- `bias/success_fail_diff_mean_{sender,receiver}`: 各世代で，成功したエピソードにおける行動の遺伝的バイアス（その行動の遺伝子の値）と失敗したエピソードにおける行動の遺伝的バイアス（その行動の遺伝子の値）との差のエピソードごとの平均．この値が大きいほど，遺伝的なバイアスが成功に寄与していることを示します．


In [ ]:
from torch.utils.tensorboard import SummaryWriter
settings = f'states{num_states}_messages{num_messages}_' \
            f'popsize{pop_size}_episodes{num_episodes_per_creature}_' \
            f'minbias{min_bias}_maxbias{max_bias}_initbias{init_bias}_' \
            f'success_fitness{success_fitness["sender"]}_{success_fitness["receiver"]}_' \
            f'success_reward{success_reward["sender"]}_{success_reward["receiver"]}_' \
            f'tornament{tornament_size}_crossover{do_crossover}_mutation{mutation_rate}_' \
            f'matching{matching_bias}_groups{num_groups}'
writer = SummaryWriter(log_dir=f'./tb/{settings}')
for i, (suc, f_m, f_v, r_ms, r_mr, g_bds, gbdr) in enumerate(zip(log_success_rate,
                                                    log_fitness_mean,
                                                    log_fitness_var,
                                                    log_rewards_mean['sender'],
                                                    log_rewards_mean['receiver'],
                                                    log_bias_suc_fail_diff_mean['sender'],
                                                    log_bias_suc_fail_diff_mean['receiver'])):
    writer.add_scalar("success/success_rate", suc, i)
    writer.add_scalar("fitness/fitness_mean", f_m, i)
    writer.add_scalar("fitness/fitness_var", f_v, i)
    writer.add_scalar("reward/sender_reward_mean", r_ms, i)
    writer.add_scalar("reward/receiver_reward_mean", r_mr, i)
    writer.add_scalar("bias/success_fail_diff_mean_sender", g_bds, i)
    writer.add_scalar("bias/success_fail_diff_mean_receiver", gbdr, i)
writer.close()

以下のコマンドでTensorboardが起動する．
このセルは一度だけ実行し，再実行しないことをお勧めする．
新しく追加したデータを再読み込みしたい場合，Tensorboardのコンソール上のリロードボタン（右上の方にあるボタン）を押してデータの再読み込みを行うようにするとよい．

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./tb

# 研究課題

1. 1世代における1個体あたりのエピソード数（`num_episodes_per_creature`）は，生涯での学習機会の量に対応する．このパラメータを変えて実験を行い，遺伝的進化の速度に与える影響を調べてみよう．学習機会が少ないほど，本能的にコミュニケーションを行うことができない遺伝子への淘汰圧は強まると考えられる．一方で，送信者と受信者の一方の本能だけではコミュニケーションは成功しないため，学習機会が全くない状況では進化が起きにくい可能性もある．Zollman & Smead (2010) https://link.springer.com/article/10.1007/s11098-009-9447-x を参照してみよう．彼らによる指摘と，実験結果との関連を考察してみよう．
2. 状態数（`num_states`）と信号数（`num_messages`）を変えて実験を行い，遺伝的進化の速度がどのように変わるか調べてみよう．状態数が多いほどコミュニケーションは成功しにくくなるため，遺伝的なバイアスが重要になり，遺伝的進化が重要になるかもしれない．一方で，状態数が多すぎると，何度やっても最適な遺伝子が見つからなくなると考えられるが，どのくらいの状態数までなら進化によって最適遺伝子が見つかるか（本能的な行動に基づくシグナリングシステムが進化できる限界の状態数はどのくらいか）について考察してみよう．
3. 集団内の個体数（`pop_size`）や，近接性の条件（`matching_bias` や `num_groups`）を変えて実験を行い，遺伝的進化の速度に与える影響を調べてみよう．個体数が多いほど，多様な遺伝子を集団内に残すことができるため，局所最適解に陥りにくくなるかもしれない．また，近接性の条件は，集団遺伝学におけるマルチレベル選択の理論（例えば，利他的行動の進化の説明）において重要な条件であるが，このこととの関連を考察してみよう．
4. 平均寿命（`lifespan_mean`）を変えて実験を行い，遺伝的進化の速度に与える影響を調べてみよう．長寿なほど多くの学習を行うことができるため，コミュニケーションの成功率は向上するはずである．しかし，長寿なほど，遺伝的なバイアスの影響が相対的に小さくなるため，遺伝的な進化を促す淘汰圧は弱まるかもしれない．
5. コミュニケーション成功時の報酬量（`success_reward`）や適応度増加量（`success_fitness`）を変えて実験を行い，遺伝的進化の速度に与える影響を調べてみよう．特に，送信者と受信者で，コミュニケーション成功時の報酬量や適応度増加量に違いがある場合，進化の過程がどのように変化するかを調べてみよう．
6. 遺伝的アルゴリズムのパラメータ（`tornament_size` や `mutation_rate`）を変えて実験を行い，遺伝的進化の速度に与える影響を調べてみよう．局所最適解に陥りにくくなるパラメータは，状態数や信号数，平均寿命などとも関連があるかもしれない．